In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Flatten, ZeroPadding2D, UpSampling2D
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import random
from tkinter import Frame, Label, CENTER

import logic
import constants as c
import math
import numpy as np
import os.path
import pandas as pd 
import random

from pynput.keyboard import Key, Controller
keyboard = Controller()



In [ ]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Flatten, ZeroPadding2D, UpSampling2D
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import gc

In [ ]:
dim = (50,50) 
def create_q_model():
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=16)

    # Convolutions on the frames on the screen
    layer1 = layers.Dense(16, activation="relu")(inputs)
    layer2 = layers.Dense(32, activation="relu")(layer1)
    layer3 = layers.Dense(64, activation="relu")(layer2)

    layer5 = layers.Dense(512, activation="relu")(layer3)
    action = layers.Dense(4, activation="linear")(layer5)

    return keras.Model(inputs=inputs, outputs=action)

In [ ]:
#model = create_q_model()
#model_json = model.to_json()
#with open(os.path.join(pathname, modeldirname,modeljsonname), "w") as json_file:
    #json_file.write(model_json)
#model.save_weights(os.path.join(pathname, modeldirname,modelweightname))
tf.enable_eager_execution() 

In [ ]:
print(gc.isenabled())

In [ ]:
#gc.set_debug(gc.DEBUG_LEAK)

class GameGrid(Frame):
    def __init__(self):
        Frame.__init__(self)

        self.grid()
        self.master.title('2048')
        self.master.bind("<Key>", self.key_down)
        self.path = "./images/data.txt"
        self.reward = 0.5
        self.punish = -0.1
        self.max = 0
        self.num = 0
        self.theaction = 0
        self.lastState = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
        self.flagBreak = 0
        self.lr=1e-3
        self.learningrate = self.lr
        self.loss_function = keras.losses.Huber()
        self.optimizer = keras.optimizers.Adam(learning_rate=self.learningrate, clipnorm=1.0)
        
        self.model = create_q_model()
        self.model_target = create_q_model()
        
        self.keys = {0:'s', 1:'a', 2:'w', 3:'d'}
        self.keysReverse = {'s':0, 'a':1, 'w':2, 'd':3}
        
        self.BATCHSIZE = 19
        self.numbatches = 0
        self.finish = False

        # self.gamelogic = gamelogic
        self.commands = {c.KEY_UP: logic.up, c.KEY_DOWN: logic.down,
                         c.KEY_LEFT: logic.left, c.KEY_RIGHT: logic.right,
                         c.KEY_UP_ALT: logic.up, c.KEY_DOWN_ALT: logic.down,
                         c.KEY_LEFT_ALT: logic.left, c.KEY_RIGHT_ALT: logic.right,
                         c.KEY_H: logic.left, c.KEY_L: logic.right,
                         c.KEY_K: logic.up, c.KEY_J: logic.down}
        
        self.grid_cells = []
        self.init_grid()
        self.init_matrix()
        self.update_grid_cells()
        self.shufflelist = []      
        
        self.load_replay_memory()
        self.mainloop()

    def init_grid(self):
        background = Frame(self, bg=c.BACKGROUND_COLOR_GAME,
                           width=c.SIZE, height=c.SIZE)
        background.grid()

        for i in range(c.GRID_LEN):
            grid_row = []
            for j in range(c.GRID_LEN):
                cell = Frame(background, bg=c.BACKGROUND_COLOR_CELL_EMPTY,
                             width=c.SIZE / c.GRID_LEN,
                             height=c.SIZE / c.GRID_LEN)
                cell.grid(row=i, column=j, padx=c.GRID_PADDING,
                          pady=c.GRID_PADDING)
                t = Label(master=cell, text="",
                          bg=c.BACKGROUND_COLOR_CELL_EMPTY,
                          justify=CENTER, font=c.FONT, width=5, height=2)
                t.grid()
                grid_row.append(t)

            self.grid_cells.append(grid_row)
            
        keyboard.press('a')
        keyboard.release('a')

    def gen(self):
        return random.randint(0, c.GRID_LEN - 1)

    def init_matrix(self):
        self.matrix = logic.new_game(c.GRID_LEN)
        self.history_matrixs = list()
        self.matrix = logic.add_two(self.matrix)
        self.matrix = logic.add_two(self.matrix)

    def update_grid_cells(self):
        for i in range(c.GRID_LEN):
            for j in range(c.GRID_LEN):
                new_number = self.matrix[i][j]
                if new_number == 0:
                    self.grid_cells[i][j].configure(
                        text="", bg=c.BACKGROUND_COLOR_CELL_EMPTY)
                else:
                    self.grid_cells[i][j].configure(text=str(
                        new_number), bg=c.BACKGROUND_COLOR_DICT[new_number],
                        fg=c.CELL_COLOR_DICT[new_number])
        self.update_idletasks()

    def writeToFile(self, key):
        liste = []
        lastStateFlat=[]
        if (self.flagBreak == 0):
            if (self.lastState != []):
                liste.append(self.num)
                self.num += 1
                for x in self.lastState:
                    for y in x:
                        if(y != 0):
                            lastStateFlat.append(int(math.log(int(y) , 2)))
                        else:
                            lastStateFlat.append(int(y))
                liste.append(lastStateFlat)
                listeNeu = []
                for x in self.matrix:
                    for y in x:
                        if(y != 0):
                            listeNeu.append(int(math.log(int(y) , 2)))
                        else:
                            listeNeu.append(int(y))
                liste.append(listeNeu)
                maximilian = 0
                maximilius = 0
                for x in self.matrix:
                    for y in x:
                        if (y == 9999999):
                            self.flagBreak = 1
                            self.reward = -1
                for x in self.lastState:
                    if(maximilian < max(x)):
                        maximilian = max(x)
                for x in self.matrix:
                    if(maximilius < max(x)):
                        maximilius = max(x)
                if(maximilius > maximilian):
                    liste.append(self.reward)
                else:
                    liste.append(self.punish)
                liste.append(key)
            else:
                liste = ["Nummer, Matrix_Alt, Matrix_Neu, reward, PressedKey"]

            f = open(self.path, "a")
            f.write(str(liste) + "\n")
            f.close()
            
            listeAlt = []
            for x in self.matrix:
                for y in x:
                    if(y != 0):
                        liste.append(int(math.log(int(y) , 2)))
                    else:
                        liste.append(int(y))
            
            self.lastState = self.matrix

    def key_down(self, event):
        #print(event)
        if event.char !='w' and event.char !='a' and event.char != 's' and event.char != 'd':
            return
        
        liste = []
        for x in self.matrix:
            for y in x:
                if(y != 0):
                    liste.append(int(math.log(int(y) , 2)))
                else:
                    liste.append(int(y))
        #print(liste)
        #print(self.lastState)
        #self.matrix = np.array(self.lastState, dtype=np.float32)
        #print(self.matrix)
        state_tensor = tf.convert_to_tensor(liste).numpy()
        state_tensor = tf.expand_dims(state_tensor, 0)
        action_probs = self.model(state_tensor, training=False)
        self.theaction = tf.argmax(action_probs[0])
        key = repr(event.char)
        if key == c.KEY_BACK and len(self.history_matrixs) > 1:
            self.matrix = self.history_matrixs.pop()
            self.update_grid_cells()
            #print('back on step total step:', len(self.history_matrixs))
        elif key in self.commands:
            self.matrix, done = self.commands[repr(event.char)](self.matrix)
            if done:
                self.matrix = logic.add_two(self.matrix)
                # record last move
                self.history_matrixs.append(self.matrix)
                self.update_grid_cells()
                done = False
                if logic.game_state(self.matrix) == 'win':
                    self.grid_cells[1][1].configure(
                        text="You", bg=c.BACKGROUND_COLOR_CELL_EMPTY)
                    self.grid_cells[1][2].configure(
                        text="Win!", bg=c.BACKGROUND_COLOR_CELL_EMPTY)
                if logic.game_state(self.matrix) == 'lose':
                    self.grid_cells[1][1].configure(
                        text="You", bg=c.BACKGROUND_COLOR_CELL_EMPTY)
                    self.grid_cells[1][2].configure(
                        text="Lose!", bg=c.BACKGROUND_COLOR_CELL_EMPTY)
                    self.matrix[0][0] = 9999999 
                    self.model_target.set_weights(self.model.get_weights())
                    self.model_target.save_weights(os.path.join(self.path))
                    for i in range(4):
                        for j in range(4):
                            self.matrix[i][j] = 0

                    #print("davor Gay grid " + str(self.matrix))
                    self.destroy()
                    self.finish = True
                    #gamegrid = GameGrid()
                    
                    #gamegrid = None
                    #quit()
                    #self.model.set_weights = self.model_target.load_weights(self.path)
        self.writeToFile(key)
        self.train()
        keyboard.press(self.generate_key())
        keyboard.release(self.generate_key())
        
        if(self.finish):
             del self
        
    def generate_key(self):
        press_rand_key = random.randint(0,10)
        if press_rand_key == 0:
            return self.keys[random.randint(0,3)]
        else:
            return self.keys[self.theaction.numpy()]

    def generate_next(self):
        index = (self.gen(), self.gen())
        while self.matrix[index[0]][index[1]] != 0:
            index = (self.gen(), self.gen())
        self.matrix[index[0]][index[1]] = 2
        
    def get_X(self, batch, state):

        assert state == 0 or state == 3 # 0 is currentstate, 3 is future state

        X = [item[state] for item in batch]

        X = np.array(X, dtype=np.float32)

        #for im in X:
        #    assert im.min() == 0.0
        #    assert im.max() == 255.0 or im.max() == 0.0

        #X /= 255.0

        return X
        
    def backprop(self, batch):

        rewards_sample = [batch[i][2] for i in range(len(batch))]
        #print("keys_reverse", self.keysReverse[batch[0][1]])
        #print("batchshape", np.array(batch))
        action_sample = [self.keysReverse[batch[i][1]] for i in range(len(batch))]
        
        #print("action_sample", action_sample)
        #print("rewards_sample", rewards_sample)
        done_sample = tf.convert_to_tensor([float(batch[i][4]) for i in range(len(batch))])
        batch = np.array(batch)
        #print(batch.shape)
        X =  self.get_X(batch, 0)
        Xf = self.get_X(batch, 3)
        #print(batch)
        future_rewards = self.model_target.predict(Xf)

        updated_q_values = rewards_sample + 0.99 * tf.reduce_max(future_rewards, axis=1)
        updated_q_values = updated_q_values * (1 - done_sample) - done_sample*abs(self.punish)

    
        masks = tf.one_hot(action_sample, 4)

        with tf.GradientTape() as tape:
            # Train the model on the states and updated Q-values
            q_values = self.model(X)

            # Apply the masks to the Q-values to get the Q-value for action taken
            q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
            # Calculate loss between new Q-value and old Q-value
            loss = self.loss_function(updated_q_values, q_action)
            
            #if self.debugcounter % 20 == 0:
            #    print(self.debugcounter)
            #    print(updated_q_values)
            #    print(rewards_sample)
            #    print(q_action)
            #self.debugcounter += 1

        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
        
 
    
    
    def train(self):
        
        # https://pythonprogramming.net/training-deep-q-learning-dqn-reinforcement-learning-python-tutorial/
        """
        currentstate = "current_{}_{}.png".format(i,j)

        nextstate = "next_{}_{}.png".format(i,j)      
        """
        batch = self.pop_batch(self.BATCHSIZE)
        #print(batch)
        """
        assert(self.imgresh1.shape == (dim[0], dim[1],3))
        assert(self.imgresh2.shape == (dim[0], dim[1],3))
        """
        #batch.append([self.imgresh1, actionstonum[self.changeto], self.reward, self.imgresh2, term, self.snake_pos[0], self.snake_pos[1], self.food_pos[0], self.food_pos[1]])
        #files.append(("current_{}_{}.png".format(i,j), "next_{}_{}.png".format(i,j)))
        
        #self.write(i,j)
         
        self.backprop(batch)
        
        self.numbatches += 1
            
        self.push_batch(batch)   
  
        return

    def push_batch(self, batch):
       
        for item in batch:
            self.shufflelist.append([item[0], item[1], item[2], item[3], item[4]])
            #self.shufflelist.append([files[index][0], item[1], item[2], files[index][1], item[4]])
    
        return


    def pop_batch(self, batchsize):
       
        batch = []
    
        for i in range(batchsize):
            
            item = self.shufflelist.pop(0)

            batch.append([item[0], item[1], item[2], item[3], item[4]])

        return batch
    
    def load_replay_memory(self):

        #assert os.path.isfile(os.path.join(self.path,datacsvname)) == True

        #f = open(os.path.join(self.path), "r")
        #print(f.readline())
        #print(f.readline())
        #df = pd.read_csv(f, index_col = 0) 

        with open(os.path.join(self.path), "r") as f:
            for line in f:
                #print (line)s[1:-1].split()
                #print(line)
                line = line.strip('][').split(', ')
                #print(self.parseThatShit(line))
                line = self.parseThatShit(line)
                currentpicname = line[1]
                action = line[4]
                reward = line[3]
                nextpicname = line[2]
            
                terminated = False
                for x in self.matrix:
                        for y in x:
                            if (y == 9999999):
                                terminated = True
                self.shufflelist.append([currentpicname,action,reward,nextpicname, terminated])
                
            f.close()
        
        """for index, row in df.iterrows():

            currentpicname = row["currentstate"]
            action = actionstonum[row["action"]]
            reward = row["reward"]
            nextpicname = row["nextstate"]
            
            terminated = False
            for x in self.matrix:
                    for y in x:
                        if (y == 9999999):
                            terminated = True
        """

                #assert os.path.isfile(os.path.join(self.path,currentpicname)) == True
                #assert (action < 5 and action >= 0)
                #assert isinstance(reward,int) or isinstance(reward, float)
                #assert os.path.isfile(os.path.join(self.path,nextpicname)) == True
            
                
            #self.shufflelist.append([currentpicname,action,reward,nextpicname, terminated])
        #print(self.shufflelist)
        random.shuffle(self.shufflelist)
        #print(f"loading: size of replay memory {len(self.shufflelist)}")
        
        #f.close()
        
        return

    def parseThatShit(self, line):
        liste = []
        listeinliste = []
        flag = 0
        for stri in line:
            if('w' in stri):
                liste.append('w')
            elif('a' in stri):
                liste.append('a')
            elif('s' in stri):
                liste.append('s')
            elif('d' in stri):
                liste.append('d')
            elif ('[' in stri):
                listeinliste.append(float(stri.split('[')[1]))
                flag = 1
            elif(']' in stri):
                #if type(stri.split(']')[0]) == float:
                if stri.split(']')[0].isnumeric():
                    listeinliste.append(float(stri.split(']')[0]))
                else:
                    listeinliste.append(stri.split(']')[0])
                flag = 0
                liste.append(listeinliste)
                listeinliste = []
            elif(flag == 1):
                listeinliste.append(float(stri))
            else:
                liste.append(float(stri))
        return liste
    
gamegrid = GameGrid()

while(True):
    print(gamegrid)
    if(gamegrid.finish == True):
        del gamegrid
        gamegrid = GameGrid()
        print("New gamegrid")